# 🚀 Day 3-1: 앙상블 기법의 정석 (Ensemble Deep-Dive)

"하나의 머리보다는 여러 개의 머리가 낫다." (Many heads are better than one.)

이 오랜 격언은 머신러닝 세계에서도 황금률과 같습니다. 

아무리 뛰어난 단일 모델이라도 예측할 수 없는 맹점(blind spot)이 존재하기 마련입니다. 

하지만 여러 모델의 '의견'을 종합하면, 개별 모델의 약점은 상쇄되고 강점은 극대화되어 훨씬 더 안정적이고 강력한 예측 성능을 이끌어낼 수 있습니다. 

이처럼 <u>여러 개의 모델(Classifier 또는 Regressor)을 결합하여 단일 모델보다 뛰어난 성능을 내는 기법</u>을 <u>앙상블(Ensemble)</u> 학습이라고 합니다.

이번 시간에는 앙상블의 가장 기본적이면서도 핵심적인 네 가지 아이디어, <u>Voting, Bagging, Boosting, Stacking</u>에 대해 깊이 있게 알아봅니다.

 각 기법이 어떤 원리로 동작하며, `scikit-learn`을 통해 어떻게 구현하는지, 그리고 이들을 통해 어떻게 모델의 <u>편향(Bias)과 분산(Variance)</u> 을 효과적으로 제어할 수 있는지 배우게 될 것입니다.

이번 실습에서는 Kaggle의 <u>심부전 예측(Heart Failure Prediction) 데이터셋</u>을 사용하여 환자의 생존 여부를 예측하는 분류 문제를 해결하며 앙상블 기법의 위력을 직접 체험해 보겠습니다.

---

### 1. 데이터 및 기본 모델 준비

앙상블 기법을 실습하기 위해, 먼저 데이터를 불러오고 앙상블에 사용할 여러 기본 모델(Base Models)을 준비하겠습니다.

#### 💻 코드로 알아보기

[Kaggle: Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)에서 `heart.csv` 파일을 다운로드하여 실습에 사용합니다.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 기본 분류 모델
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# 앙상블 모델
from sklearn.ensemble import VotingClassifier, BaggingClassifier, StackingClassifier, AdaBoostClassifier

# 평가 지표
from sklearn.metrics import accuracy_score, roc_auc_score

# 데이터셋 로드 (자신의 파일 경로에 맞게 수정해주세요)
path = '../datasets/ml/heart-failure/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(path)
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
# 특성과 타겟 분리
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# 전처리 파이프라인 설정
numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

numeric_feature_indices = [X.columns.get_loc(col) for col in numeric_features]
categorical_feature_indices = [X.columns.get_loc(col) for col in categorical_features]


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indices), # 인덱스 값으로 지정
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_feature_indices)
    ])

print("데이터 준비 완료!")
print(f"훈련 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

데이터 준비 완료!
훈련 데이터 크기: (239, 12)
테스트 데이터 크기: (60, 12)


---

### 2. 투표 (Voting)

가장 직관적인 앙상블 방법입니다. 여러 명의 전문가가 투표를 통해 최종 결정을 내리는 것과 같습니다. 각 모델이 독립적으로 예측한 결과를 모아, 다수결 또는 평균으로 최종 결론을 내립니다.

#### 🧠 개념 이해하기

* <u>하드 보팅 (Hard Voting)</u>: 다수결 원칙입니다. 여러 모델 중 가장 많은 표를 받은 클래스를 최종 예측값으로 선택합니다. '분류(Classification)' 문제에만 사용할 수 있습니다.
    * 예: 모델 A가 '생존', 모델 B가 '사망', 모델 C가 '생존'으로 예측했다면, 최종 예측은 '생존'이 됩니다.

* <u>소프트 보팅 (Soft Voting)</u>: 각 모델이 예측한 '확률'을 평균 내어, 가장 높은 확률을 가진 클래스를 최종 예측값으로 선택합니다. 일반적으로 하드 보팅보다 성능이 더 좋다고 알려져 있습니다. 각 모델의 예측 신뢰도를 반영할 수 있기 때문입니다.
    * 예: 모델 A가 '생존' 확률 80%, 모델 B가 40%, 모델 C가 60%로 예측했다면, 평균 '생존' 확률은 $(0.8+0.4+0.6)/3 = 0.6$ 즉 60%가 되어 최종 예측은 '생존'이 됩니다.
    * 모델에 <u>가중치(weights)</u>를 부여하여 특정 모델의 의견에 더 힘을 실어줄 수도 있습니다.

#### 💻 코드로 알아보기

로지스틱 회귀, K-최근접 이웃, 결정 트리 세 가지 모델을 사용하여 `VotingClassifier`를 만들어 보겠습니다.

In [8]:
# 1. 앙상블에 사용할 기본 모델 정의
# 각 모델을 파이프라인으로 묶어 전처리가 자동으로 적용되도록 합니다.
lr_clf = Pipeline([('preprocessor', preprocessor), ('classifier', LogisticRegression(random_state=42))])
knn_clf = Pipeline([('preprocessor', preprocessor), ('classifier', KNeighborsClassifier(n_neighbors=5))])
dt_clf = Pipeline([('preprocessor', preprocessor), ('classifier', DecisionTreeClassifier(random_state=42))])

# 2. 하드 보팅 (Hard Voting)
# 'estimators'에는 (모델명, 모델객체) 튜플의 리스트를 전달
voting_clf_hard = VotingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='hard'
)

# 3. 소프트 보팅 (Soft Voting)
# voting='soft'로 변경
voting_clf_soft = VotingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='soft'
)

# 모델 학습
lr_clf.fit(X_train, y_train)
knn_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
voting_clf_hard.fit(X_train, y_train)
voting_clf_soft.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, f1_score

# 각 모델의 정확도 평가
models = [('Logistic Regression', lr_clf), ('KNN', knn_clf), ('Decision Tree', dt_clf),
          ('Hard Voting', voting_clf_hard), ('Soft Voting', voting_clf_soft)]

for name, model in models:
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1_score_ = f1_score(y_test, y_pred)
    print(f'{name} Accuracy: {accuracy:.4f}')
    print(f'{name} f1_score: {f1_score_:.4f}')

Logistic Regression Accuracy: 0.8167
Logistic Regression f1_score: 0.6667
KNN Accuracy: 0.7000
KNN f1_score: 0.3077
Decision Tree Accuracy: 0.7333
Decision Tree f1_score: 0.5294
Hard Voting Accuracy: 0.7833
Hard Voting f1_score: 0.5806
Soft Voting Accuracy: 0.7500
Soft Voting f1_score: 0.5455


#### ✏️ 연습문제 1

소프트 보팅 시, `DecisionTreeClassifier` 모델의 성능이 다른 모델보다 상대적으로 낮다고 가정해봅시다. `VotingClassifier`의 `weights` 파라미터를 사용하여 **로지스틱 회귀에 2, KNN에 2, 결정 트리에 1의 가중치**를 부여한 새로운 소프트 보팅 모델(`voting_clf_weighted`)을 만들고, 정확도를 평가해보세요.

In [ ]:
# 연습문제 1 코드

# 가중치를 적용한 소프트 보팅 모델을 정의하세요.
# 연습문제 1: 소프트 보팅에 가중치 적용
voting_clf_weighted = VotingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='soft',
    weights=[2, 2, 1]
)

# 모델 학습
voting_clf_weighted.fit(X_train, y_train)
weighted_accuracy = voting_clf_weighted.score(X_test, y_test)

# 정확도 평가
print(f'Weighted Soft Voting Accuracy: {weighted_accuracy:.4f}')

Weighted Soft Voting Accuracy: 0.7667


---

### 3. 배깅 (Bagging)

<u>배깅(Bagging)</u>은 <u>B</u>ootstrap <u>Agg</u>regat<u>ing</u>의 약자로, 같은 알고리즘을 사용하지만 <u>서로 다른 훈련 데이터 샘플</u>로 여러 모델을 학습시켜 그 결과를 종합하는 기법입니다.

#### 🧠 개념 이해하기

1.  <u>부트스트랩 (Bootstrap)</u>: 원본 훈련 데이터에서 <u>중복을 허용하여</u> 원본 데이터와 같은 크기의 샘플을 여러 개 만드는 과정입니다. (예: [A, B, C, D]에서 샘플링하면 [A, B, B, D]가 나올 수 있습니다.)
2.  <u>병렬 학습 (Parallel Training)</u>: 각 부트스트랩 샘플을 사용하여 독립적으로 여러 개의 모델을 <u>동시에(병렬로)</u> 학습시킵니다.
3.  <u>결과 종합 (Aggregating)</u>: 각 모델의 예측 결과를 투표(Voting)를 통해 합칩니다.

배깅은 각 모델이 약간씩 다른 데이터를 학습하기 때문에, 개별 모델의 예측이 조금씩 달라집니다. 이들의 예측을 평균내면 한쪽으로 치우친 예측(과대적합)을 줄일 수 있어 <u>모델의 분산(Variance)을 낮추는 효과</u>가 매우 큽니다.

가장 대표적인 배깅 기반 알고리즘이 바로 <u>랜덤 포레스트(Random Forest)</u>입니다. 랜덤 포레스트는 결정 트리(Decision Tree)를 기본 모델로 사용하며, 부트스트랩뿐만 아니라 <u>각 노드에서 특성을 무작위로 선택</u>하는 과정을 추가하여 모델 간의 상관관계를 더욱 낮춘, 배깅의 확장판입니다.

#### 💻 코드로 알아보기

`scikit-learn`의 `BaggingClassifier`를 사용해 결정 트리 모델을 배깅 방식으로 앙상블 해보겠습니다.

In [ ]:
# BaggingClassifier 사용
# estimator: 기본 모델
# n_estimators: 앙상블할 모델의 개수
# max_samples: 각 모델이 학습할 샘플의 비율
# n_jobs=-1: 모든 CPU 코어를 사용하여 병렬 학습
bagging_clf = BaggingClassifier(
    estimator=Pipeline([('preprocessor', preprocessor),
                       ('classifier', DecisionTreeClassifier(random_state=42))]),
    n_estimators=50,
    max_samples=0.8, # 훈련 데이터의 80%를 샘플링
    random_state=42,
    n_jobs=-1
)

# Bagging 모델 학습
bagging_clf.fit(X_train, y_train)

# 정확도 평가
bagging_accuracy = bagging_clf.score(X_test, y_test)
print(f'Bagging (Decision Tree) Accuracy: {bagging_accuracy:.4f}')

# 참고: 단일 결정 트리 모델의 정확도
dt_accuracy = dt_clf.score(X_test, y_test)
print(f'Single Decision Tree Accuracy: {dt_accuracy:.4f}')

Bagging (Decision Tree) Accuracy: 0.8167
Single Decision Tree Accuracy: 0.7333


#### ✏️ 연습문제 2

`BaggingClassifier`의 기본 모델(`base_estimator`)을 `LogisticRegression`으로 변경하여 새로운 배깅 모델(`bagging_lr_clf`)을 만들어 보세요. `n_estimators`는 30개로 설정하고, 학습 후 정확도를 평가하여 출력하세요.

In [ ]:
# 연습문제 2 코드

# 로지스틱 회귀를 기본 모델로 사용하는 BaggingClassifier를 정의하세요.
# 연습문제 2: BaggingClassifier의 estimator를 LogisticRegression으로 변경
bagging_lr_clf = BaggingClassifier(
    estimator=Pipeline([('preprocessor', preprocessor),
                             ('classifier', LogisticRegression(random_state=42))]),
    n_estimators=30,
    random_state=42,
    n_jobs=-1
)

# 모델 학습
bagging_lr_clf.fit(X_train, y_train)
lr_bagging_accuracy = bagging_lr_clf.score(X_test, y_test)

# 정확도 평가
print(f'Bagging (Logistic Regression) Accuracy: {lr_bagging_accuracy:.4f}')

---

### 4. 부스팅 (Boosting)

<u>부스팅(Boosting)</u>은 여러 개의 약한 학습기(Weak Learner)를 <u>순차적으로</u> 학습시켜, 이전 모델이 잘못 예측한 데이터에 가중치를 부여하며 점점 더 강력한 모델을 만들어가는 기법입니다.

#### 🧠 개념 이해하기

1.  첫 번째 모델이 전체 데이터를 학습하고 예측합니다.
2.  <u>예측이 틀린 데이터에 가중치</u>를 높입니다. (더 중요하게 만듭니다.)
3.  두 번째 모델은 가중치가 적용된 데이터를 학습합니다. 즉, 이전 모델이 어려워했던 문제에 더 집중하게 됩니다.
4.  이 과정을 반복하며, 각 모델은 이전 모델의 실수를 보완하는 방향으로 학습됩니다.
5.  최종 예측은 모든 모델의 예측을 가중합하여 결정합니다.

부스팅은 잘못된 부분을 집중적으로 개선해 나가기 때문에 <u>모델의 편향(Bias)을 낮추는 데</u> 매우 효과적입니다. 대표적인 부스팅 알고리즘으로는 <u>AdaBoost, Gradient Boosting, XGBoost, LightGBM</u> 등이 있습니다.

#### 💻 코드로 알아보기

가장 기본적인 부스팅 알고리즘 중 하나인 `AdaBoostClassifier`를 사용해 보겠습니다. AdaBoost는 결정 트리를 기본 모델로 사용합니다.

In [23]:
# AdaBoostClassifier 사용
# AdaBoost는 내부적으로 전처리를 직접 처리하기 어려우므로,
# 전처리된 데이터를 직접 입력해야 합니다.
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# AdaBoost 모델 정의
adaboost_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=50,
    learning_rate=0.5,
    random_state=42
)

# AdaBoost 모델 학습
adaboost_clf.fit(X_train_processed, y_train)

# 정확도 평가
adaboost_accuracy = adaboost_clf.score(X_test_processed, y_test)
print(f'AdaBoost Accuracy: {adaboost_accuracy:.4f}')

AdaBoost Accuracy: 0.7667


#### ✏️ 연습문제 3

`AdaBoostClassifier`에서 학습률(`learning_rate`)은 각 모델이 이전 모델의 오류를 얼마나 강하게 보정할지를 결정하는 중요한 파라미터입니다. `learning_rate`를 `1.0`으로 변경하여 새로운 AdaBoost 모델(`adaboost_clf_lr1`)을 만들고, 학습 후 정확도를 평가해보세요.

In [ ]:
# 연습문제 3 코드

# learning_rate가 1.0인 AdaBoost 모델을 정의하세요.
# 연습문제 3: AdaBoostClassifier의 learning_rate를 1.0으로 변경
adaboost_clf_lr1 = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=50,
    learning_rate=1.0,
    random_state=42
)

# 모델 학습
adaboost_clf_lr1.fit(X_train_processed, y_train)
ada_lr1_accuracy = adaboost_clf_lr1.score(X_test_processed, y_test)

# 정확도 평가
print(f'AdaBoost (learning_rate=1.0) Accuracy: {ada_lr1_accuracy:.4f}')

---

### 5. 스태킹 (Stacking)

<u>스태킹(Stacking)</u>은 여러 다른 종류의 모델(Base Models)의 예측값을 <u>특성(feature)처럼</u> 사용하여, 이 예측값들을 학습하는 새로운 모델(Meta Model)을 통해 최종 예측을 수행하는 복잡하고 강력한 앙상블 기법입니다.

#### 🧠 개념 이해하기

1.  <u>Level 0 (Base Models)</u>: 여러 개의 기본 모델(e.g., 로지스틱 회귀, KNN, 결정 트리)이 훈련 데이터를 학습하고 예측값을 만듭니다.
2.  <u>Level 1 (Meta Model)</u>: Level 0에서 생성된 예측값들을 <u>입력 특성</u>으로 받아서 학습하는 최종 모델입니다. 이 메타 모델은 각 기본 모델의 예측 결과를 어떻게 조합해야 최적의 결과를 낼 수 있는지를 학습합니다.

스태킹은 마치 "각 분야 전문가(Base Models)들의 소견을 듣고 최종 결정을 내리는 관리자(Meta Model)"와 같습니다. 서로 다른 방식의 모델들을 조합하여 각 모델의 장점만을 취하려는 시도이기 때문에, 종종 캐글(Kaggle)과 같은 데이터 경진대회에서 높은 성능을 달성하기 위해 사용됩니다.

<u>중요</u>: 데이터 유출(Data Leakage)을 방지하기 위해, 스태킹에서는 <u>교차 검증(Cross-Validation)</u> 방식을 사용하여 기본 모델들이 예측값을 생성하도록 하는 것이 일반적입니다.

#### 💻 코드로 알아보기

`scikit-learn`의 `StackingClassifier`를 사용하여 로지스틱 회귀, KNN, 결정 트리를 기본 모델로, 최종 메타 모델은 로지스틱 회귀를 사용하는 스태킹 앙상블을 구축해 보겠습니다.

In [24]:
# StackingClassifier 사용
# estimators: 기본 모델들 (Voting과 동일한 형식)
# final_estimator: 메타 모델
# cv: 데이터 유출 방지를 위한 교차 검증 폴드 수
stacking_clf = StackingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    final_estimator=LogisticRegression(random_state=42),
    cv=5,
    n_jobs=-1
)

# Stacking 모델 학습
stacking_clf.fit(X_train, y_train)

# 정확도 평가
stacking_accuracy = stacking_clf.score(X_test, y_test)
print(f'Stacking Accuracy: {stacking_accuracy:.4f}')

Stacking Accuracy: 0.8000


#### ✏️ 연습문제 4

`StackingClassifier`의 메타 모델(`final_estimator`)을 `DecisionTreeClassifier(max_depth=2, random_state=42)`로 변경하여 새로운 스태킹 모델(`stacking_dt_clf`)을 만들고, 학습 후 정확도를 평가해보세요.

In [ ]:
# 연습문제 4 코드

# 메타 모델이 DecisionTree인 StackingClassifier를 정의하세요.
stacking_dt_clf = StackingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    final_estimator=?, # 여기에 DecisionTreeClassifier를 넣으세요.
    cv=5,
    n_jobs=-1
)

# 모델 학습
# ?

# 정확도 평가
# ?
# print(f'Stacking (Meta=DT) Accuracy: {stacking_dt_accuracy:.4f}')